## 참고 : SVD 로 평점예측

https://data-science-hi.tistory.com/82

해당 SVD 에서는 아이템 별 평균을 구한 다음 사용자 별 평균을 빼주고, SVD 분해 이후 다시 사용자 별 평균을 더해준다.

In [28]:
import HA_ndcg

## 데이터 LOAD

In [1]:
import pandas as pd
import numpy as np
import math

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')

#컬럼명 수정
ratings.columns = ['userid','movieid','rating','timestamp']

#불필요한 컬럼 제거
ratings = ratings.drop(['timestamp'], axis=1)

#피벗테이블로 만들어서 사용자-아이템 행렬 확인하기
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])
rating_table

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# 80000 row
user_train = pd.read_csv('../ml-100k_GRS/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]

# 20000 row
user_test = pd.read_csv('../ml-100k_GRS/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]

In [20]:
user_item_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_item_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

## 2. SVD 로 평점 예측

In [2]:
def R_filled_in(rating_table):
    for col in range(len(rating_table.columns)):
        col_update=[]
        # 컬럼의 평균을 구한다.
        col_num = [i for i in rating_table.iloc[:,col] if math.isnan(i)==False]
        col_mean = sum(col_num)/len(col_num)
        
        # NaN을 가진 행은 위에서 구한 평균 값으로 채워준다.
        col_update = [i if math.isnan(i)==False else col_mean for i in rating_table.iloc[:,col]]

        # 리스트로 만든 업데이트된 한 컬럼을 기존에 데이터 프레임 컬럼에 새로 입혀준다.
        rating_table.iloc[:,col] = col_update
        
    return rating_table
    

In [3]:
rating_R_filled = R_filled_in(rating_table)
rating_R_filled

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
2,4.000000,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,2.000000,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
3,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
4,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
5,4.000000,3.000000,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,5.000000,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
940,3.878319,3.206107,3.033333,2.000000,3.302326,3.576923,4.000000,5.000000,3.000000,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
941,5.000000,3.206107,3.033333,3.550239,3.302326,3.576923,4.000000,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0


In [13]:
def R_norm(rating_R_filled):
    for row in range(len(rating_R_filled)):
        # 각 값에 평균을 빼준 값들을 넣을 리스트 공간 생성.
        row_update=[]

        # 행 평균 값
        row_mean=sum(rating_R_filled.iloc[row,:])/len(rating_R_filled.iloc[row,:])

        # 해당 행의 모든 컬럼 값에 행 평균 값을 뺀다.
        row_update= [i - row_mean for i in rating_R_filled.iloc[row,:]]

        rating_R_filled.iloc[row,:] = row_update

    return rating_R_filled, row_mean

rating_R_norm, row_mean = R_norm(rating_R_filled)

In [14]:
from sklearn.decomposition import TruncatedSVD

# TruncatedSVD를 사용해서 차원축소를 해보자. 여기서 k는 12로 두었다. n_iter는 랜덤 SVD 계산기의 반복횟수이다.
svd = TruncatedSVD(n_components=12, n_iter=5)

# 모델을 데이터에 맞추는 작업
svd.fit(np.array(rating_R_norm))

# 다음의 코드를 통해 각각 행렬 U, S, V^T 를 변수에 담아낼 수 있다.
U=svd.fit_transform(np.array(rating_R_norm))
Sigma=svd.explained_variance_ratio_
VT= svd.components_

In [15]:
# 시그마 행렬에 요소들에 루트를 씌워주는 작업.
Sigma_sqrt=[]
for i in range(len(Sigma)):
    tmp=[]
    tmp=[math.sqrt(s) for s in np.diag(Sigma)[i]]
    Sigma_sqrt.append(tmp)
        
Sigma_sqrt=np.array(Sigma_sqrt)    


# 행렬 곱
#pd.DataFrame(np.matmul(U, Sigma_sqrt))

# 행렬곱
#pd.DataFrame(np.matmul(Sigma_sqrt, VT))

In [16]:
ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT))
ratings_reduced

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.004485,0.001974,0.000502,0.009181,-0.002628,0.004290,0.013667,0.006797,0.009773,0.005982,...,-0.001137,0.005265,-0.001137,-0.007538,-0.001137,-0.013939,-0.001137,-0.007538,-0.001137,-0.001137
1,0.004859,0.000377,-0.000298,0.004071,0.001239,0.003878,0.005182,0.006313,0.007197,0.005239,...,-0.000563,0.006086,-0.000563,-0.007213,-0.000563,-0.013862,-0.000563,-0.007213,-0.000563,-0.000563
2,0.002203,0.000875,-0.001156,0.003467,-0.000110,0.002642,0.001668,0.006848,0.005014,0.004823,...,-0.000181,0.006652,-0.000181,-0.007013,-0.000181,-0.013846,-0.000181,-0.007013,-0.000181,-0.000181
3,0.008013,0.000746,0.000172,0.002738,0.002297,0.003967,0.007140,0.005589,0.005483,0.005629,...,-0.000661,0.006090,-0.000661,-0.007411,-0.000661,-0.014162,-0.000661,-0.007411,-0.000661,-0.000661
4,-0.003525,-0.001312,-0.000566,0.002002,-0.001743,0.005410,0.008229,0.001556,0.003703,0.007460,...,0.000008,0.007250,0.000008,-0.007234,0.000008,-0.014476,0.000008,-0.007234,0.000008,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.011986,0.003387,0.002101,0.003913,0.004293,0.003069,0.009292,0.007731,0.004334,0.005043,...,-0.001148,0.005417,-0.001148,-0.007713,-0.001148,-0.014279,-0.001148,-0.007713,-0.001148,-0.001148
939,0.005037,0.001185,-0.000145,0.000496,0.002653,0.003167,0.004324,0.004645,0.002167,0.005062,...,-0.000407,0.006503,-0.000407,-0.007316,-0.000407,-0.014225,-0.000407,-0.007316,-0.000407,-0.000407
940,0.008291,0.001338,0.000295,0.003624,0.001657,0.003745,0.005936,0.006495,0.005616,0.005395,...,-0.000734,0.005944,-0.000734,-0.007412,-0.000734,-0.014090,-0.000734,-0.007412,-0.000734,-0.000734
941,0.011345,0.002403,-0.000217,0.004238,0.002531,0.002929,0.003817,0.007935,0.004884,0.003693,...,-0.000907,0.005582,-0.000907,-0.007396,-0.000907,-0.013885,-0.000907,-0.007396,-0.000907,-0.000907


해당 수치에 처음 뺐던 사용자 별 평균을 다시 더해준다.


In [17]:
def R_pred(ratings_reduced, row_mean):
    for row in range(len(ratings_reduced)):
        # 각 값에 평균을 더해 준 값들을 넣을 리스트 공간 생성.
        row_update=[]

        # 해당 행의 모든 컬럼 값에 행 평균 값을 뺀다.
        row_update= [i + row_mean for i in ratings_reduced.iloc[row,:]]

        ratings_reduced.iloc[row,:] = row_update

    return ratings_reduced

R = R_pred(ratings_reduced, row_mean)

R


,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.004485,0.001974,0.000502,0.009181,-0.002628,0.004290,0.013667,0.006797,0.009773,0.005982,...,-0.001137,0.005265,-0.001137,-0.007538,-0.001137,-0.013939,-0.001137,-0.007538,-0.001137,-0.001137
1,0.004859,0.000377,-0.000298,0.004071,0.001239,0.003878,0.005182,0.006313,0.007197,0.005239,...,-0.000563,0.006086,-0.000563,-0.007213,-0.000563,-0.013862,-0.000563,-0.007213,-0.000563,-0.000563
2,0.002203,0.000875,-0.001156,0.003467,-0.000110,0.002642,0.001668,0.006848,0.005014,0.004823,...,-0.000181,0.006652,-0.000181,-0.007013,-0.000181,-0.013846,-0.000181,-0.007013,-0.000181,-0.000181
3,0.008013,0.000746,0.000172,0.002738,0.002297,0.003967,0.007140,0.005589,0.005483,0.005629,...,-0.000661,0.006090,-0.000661,-0.007411,-0.000661,-0.014162,-0.000661,-0.007411,-0.000661,-0.000661
4,-0.003525,-0.001312,-0.000566,0.002002,-0.001743,0.005410,0.008229,0.001556,0.003703,0.007460,...,0.000008,0.007250,0.000008,-0.007234,0.000008,-0.014476,0.000008,-0.007234,0.000008,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.011986,0.003387,0.002101,0.003913,0.004293,0.003069,0.009292,0.007731,0.004334,0.005043,...,-0.001148,0.005417,-0.001148,-0.007713,-0.001148,-0.014279,-0.001148,-0.007713,-0.001148,-0.001148
939,0.005037,0.001185,-0.000145,0.000496,0.002653,0.003167,0.004324,0.004645,0.002167,0.005062,...,-0.000407,0.006503,-0.000407,-0.007316,-0.000407,-0.014225,-0.000407,-0.007316,-0.000407,-0.000407
940,0.008291,0.001338,0.000295,0.003624,0.001657,0.003745,0.005936,0.006495,0.005616,0.005395,...,-0.000734,0.005944,-0.000734,-0.007412,-0.000734,-0.014090,-0.000734,-0.007412,-0.000734,-0.000734
941,0.011345,0.002403,-0.000217,0.004238,0.002531,0.002929,0.003817,0.007935,0.004884,0.003693,...,-0.000907,0.005582,-0.000907,-0.007396,-0.000907,-0.013885,-0.000907,-0.007396,-0.000907,-0.000907


---

## 3. 클러스터링

In [21]:
HA_ndcg.groupRS_ndcg(3,R, user_item_train, user_item_test)

각 cluster 별 user 회원 수 :  [109, 74, 760]
cluster수: 3 / NDCG: 0.2449540627366976
0.24495


In [30]:
cluster_num = range(2,10)

for i in cluster_num:
    HA_ndcg.groupRS_ndcg(i,R, user_item_train, user_item_test)

각 cluster 별 user 회원 수 :  [183, 760]
cluster수: 2 / NDCG: 0.2356611791829788
0.23566
각 cluster 별 user 회원 수 :  [107, 74, 762]
cluster수: 3 / NDCG: 0.24302882124483913
0.24303
각 cluster 별 user 회원 수 :  [749, 122, 1, 71]
cluster수: 4 / NDCG: 0.23137223172231497
0.23137
각 cluster 별 user 회원 수 :  [32, 1, 81, 593, 236]
cluster수: 5 / NDCG: 0.23544500134493918
0.23545
각 cluster 별 user 회원 수 :  [581, 48, 1, 128, 23, 162]
cluster수: 6 / NDCG: 0.2352491118571778
0.23525
각 cluster 별 user 회원 수 :  [170, 122, 23, 1, 578, 48, 1]
cluster수: 7 / NDCG: 0.24965649808659743
0.24966
각 cluster 별 user 회원 수 :  [165, 48, 19, 533, 90, 1, 86, 1]
cluster수: 8 / NDCG: 0.25382568456474214
0.25383
각 cluster 별 user 회원 수 :  [564, 38, 21, 1, 40, 123, 1, 1, 154]
cluster수: 9 / NDCG: 0.2576139832614103
0.25761


In [26]:
HA_ndcg.groupRS_ndcg(20,R, user_item_train, user_item_test)

각 cluster 별 user 회원 수 :  [51, 57, 1, 15, 31, 234, 23, 27, 16, 7, 1, 1, 6, 4, 32, 251, 71, 57, 42, 16]
cluster수: 20 / NDCG: 0.24654531710472885
0.24655


In [29]:
HA_ndcg.groupRS_ndcg(20,R, user_item_train, user_item_test)

각 cluster 별 user 회원 수 :  [92, 15, 42, 1, 18, 38, 40, 25, 170, 206, 107, 5, 1, 16, 1, 53, 27, 8, 7, 71]
cluster수: 20 / NDCG: 0.24800205587342927
0.24800
